# 02_entiry-extract

In [ ]:
!pip install transformers torch
!pip install sentencepiece

# install extra library

In [ ]:
from dsmlibrary.datanode import DataNode
import json
from tqdm.auto import tqdm
import pandas as pd
import re

## read stored data from DataPlatform

In [ ]:
dir_raw_id = 62
dir_process_id = 63

In [ ]:
datanode = DataNode()

In [ ]:
meta, fp = datanode.get_file(datanode.get_file_id(directory_id=dir_raw_id, name='events_jewelrymagazine.json'))
meta

In [ ]:
data = json.load(fp)
data

# NER (NameEntitiesRegcognition)

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
model = AutoModelForTokenClassification.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

In [ ]:
example = "Five Crystals You Need To Bring In Wealth, Prosperity, and Success Into Your Life"

ner_results = nlp(example)
print(ner_results)

In [ ]:
def post_process(preds):
    data = []
    for i in range(len(preds)): 
        a = preds[i]
        text = a.get('word')
        for j in range(i, len(preds)):
            b = preds[j]
            if b.get('start') == a.get('end'):
                text += b.get('word')
                a = b
        data.append({
            'type': a.get('entity'),
            'text': text
        })
    tmp = []
    for elm in data:
        text = elm.get('text')
        if bool(re.match("[ก-๛]|▁", text)):
            elm.update({'text': text.replace('▁', ' ')})
            tmp.append(elm)
    return tmp

In [ ]:
text = data[5]['Description']
text

In [ ]:
output = nlp(text)
output

In [ ]:
post_process(output)

# Process events_jewelrymagazine to spo

In [ ]:
spo = []
for events_jewelrymagazine in tqdm(data):
    Title = events_jewelrymagazine.get('Title') 
    if Title == None:
        continue
    ner = nlp(Title)
    processed = post_process(ner)
    for kw in processed:
        spo.append({
            'Title': events_jewelrymagazine.get('Title', ""),
            'Publish_time': events_jewelrymagazine.get('Publish_time', ""),
            'Description': events_jewelrymagazine.get('Description', ""),
            'Link': events_jewelrymagazine.get('Link', "")
        })

In [ ]:
df = pd.DataFrame(spo)
df['Title'] = df['Title'].apply(lambda x: x[2:])
df

In [ ]:
datanode.write(df=df, directory=dir_process_id, name="spo", profiling=True, lineage=[datanode.get_file_id(directory_id=dir_raw_id, 
                                                                                                          name='events_jewelrymagazine.json')])